On every epoch end

In [ ]:
class CustomCallbackHM(Callback):  
    # Stop the algorithm when the following accuracy reached 
    stop_at = 0.993  
    every = 13
    counter = 1 
    num_layers = 0
    previous_weights = 0
    call_hm = 0   
    
    def on_train_begin(self, logs=None):
        # Harmonic mean based weights calculation method invocation
        self.call_hm = np.vectorize(self.apply_hm)   
        self.num_layers = len(model_hm.layers)

    def on_train_batch_end(self, epoch, logs={}):         
        # To access the weights of a model at second last step
        if self.counter == self.every-1:
          self.previous_weights = model_hm.get_weights() 
          self.previous_weights = np.array(self.previous_weights, dtype=object) 
        self.counter = self.counter + 1
        
    def on_epoch_end(self, epoch, logs={}): 
        # Training stop criteria
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True
            
        # To access the weights of a model at last step 
        current_weights = model_hm.get_weights() 
        current_weights = np.array(current_weights, dtype=object)     
            
        for i in range(self.num_layers):  
            # Harmonic mean based weights calculation
            tensor1 = tf.convert_to_tensor(self.previous_weights[i*2])
            tensor2 = tf.convert_to_tensor(current_weights[i*2])
            current_weights[i*2] = self.call_hm(tensor1, tensor2)   
            
        # Updating the model with new weights
        model_hm.set_weights(current_weights.tolist()) 
        self.counter = 1 
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  

on every specific set of batches

In [ ]:
class CustomCallbackHM(Callback):  
    # Stop the algorithm when the following accuracy reached 
    stop_at = 0.995  
    every = 300
    counter = 1
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 

    def on_train_batch_end(self, epoch, logs={}): 
        if self.counter == self.every-1:
          self.initial_weights = model_hm.get_weights() 
          self.initial_weights = np.array(self.initial_weights, dtype=object) 
          # Harmonic mean based weights calculation
          self.call_hm = np.vectorize(self.apply_hm)  

        if self.counter % self.every == 0:   
          current_weights = model_hm.get_weights() 
          current_weights = np.array(current_weights, dtype=object)      

          tensor1 = tf.convert_to_tensor(self.initial_weights[4])
          tensor2 = tf.convert_to_tensor(current_weights[4])
          current_weights[4] = self.call_hm(tensor1, tensor2)

          tensor1 = tf.convert_to_tensor(self.initial_weights[6])
          tensor2 = tf.convert_to_tensor(current_weights[6])
          current_weights[6] = self.call_hm(tensor1, tensor2) 

          tensor1 = tf.convert_to_tensor(self.initial_weights[8])
          tensor2 = tf.convert_to_tensor(current_weights[8])
          current_weights[8] = self.call_hm(tensor1, tensor2)        
               
          # Updating the model with new weights   
          model_hm.set_weights(current_weights.tolist())
        self.counter = self.counter + 1

    def on_epoch_end(self, epoch, logs=None): 
        self.counter = 1
        # Training stop criteria
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True

    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  

In [4]:
0 < 0

False